In [5]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import csv
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

# The caffe module needs to be on the Python path;
#  we'll add it here explicitly.
import sys
caffe_root = '/home/ambika/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe
# If you get "No module named _caffe", either you have not built pycaffe or you have the wrong path.

import os
if os.path.isfile(caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print 'CaffeNet found.'
else:
    print 'Downloading pre-trained CaffeNet model...'
    !../scripts/download_model_binary.py ../models/bvlc_reference_caffenet
    
caffe.set_mode_cpu()

model_def = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
model_weights = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)

# load the mean ImageNet image (as distributed with Caffe) for subtraction
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# create transformer for the input called 'data'
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

# set the size of the input (we can skip this if we're happy
#  with the default; we can also change it later, e.g., for different batch sizes)
net.blobs['data'].reshape(50,        # batch size
                          3,         # 3-channel (BGR) images
                          227, 227)  # image size is 227x227



CaffeNet found.
mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]


In [6]:
final_path='/home/ambika/INF_project/final_image_paths.txt'
image_list = np.loadtxt(final_path, str)

#image = caffe.io.load_image(caffe_root + 'examples/images/cat.jpg')
# load ImageNet labels
labels_file = caffe_root + 'data/ilsvrc12/synset_words.txt'
if not os.path.exists(labels_file):
    !../data/ilsvrc12/get_ilsvrc_aux.sh
    
labels = np.loadtxt(labels_file, str, delimiter='\t')
filename='alexnet_labels_1000.txt'
b = open(filename, 'wb') 
a = csv.writer(b)

for i in range(0,1000):
    #image = caffe.io.load_image('/home/ambika/INF_project/data/dog/166995.jpg')
    print i
    image = caffe.io.load_image(image_list[i])
    transformed_image = transformer.preprocess('data', image)
    #copy the image data into the memory allocated for the net
    net.blobs['data'].data[...] = transformed_image

    ### perform classification
    output = net.forward()

    output_prob = output['prob'][0]  # the output probability vector for the first image in the batch

    print 'predicted class is:', output_prob.argmax()
    print 'output label:', labels[output_prob.argmax()]
    # sort top five predictions from softmax output
    top_inds = output_prob.argsort()[::-1][:5]  # reverse sort and take five largest items

    print 'probabilities and labels:'
    print zip(output_prob[top_inds], labels[top_inds])
    
    
    data=str(output_prob[top_inds[0]])+'\t'+labels[top_inds[0]]+'\t'+str(output_prob[top_inds[1]])+'\t'+labels[top_inds[1]]+'\t'+str(output_prob[top_inds[2]])+'\t'+labels[top_inds[2]]+'\t'+str(output_prob[top_inds[3]])+'\t'+labels[top_inds[3]]+'\t'+str(output_prob[top_inds[4]])+'\t'+labels[top_inds[4]]
    new_data=[[image_list[i]+'\t'+data]]
    a.writerows(new_data)
    #plt.imshow(image)
    
b.close()

0
predicted class is: 908
output label: n04592741 wing
probabilities and labels:
[(0.42647114, 'n04592741 wing'), (0.28220782, 'n02690373 airliner'), (0.27878064, 'n04552348 warplane, military plane'), (0.003935222, 'n04266014 space shuttle'), (0.0022067756, 'n03773504 missile')]
1
predicted class is: 827
output label: n04330267 stove
probabilities and labels:
[(0.059061941, 'n04330267 stove'), (0.033070184, 'n04442312 toaster'), (0.031779107, 'n04604644 worm fence, snake fence, snake-rail fence, Virginia fence'), (0.029097266, 'n04590129 window shade'), (0.0266473, 'n03207941 dishwasher, dish washer, dishwashing machine')]
2
predicted class is: 675
output label: n03796401 moving van
probabilities and labels:
[(0.23737109, 'n03796401 moving van'), (0.19835046, 'n03417042 garbage truck, dustcart'), (0.11967088, 'n03345487 fire engine, fire truck'), (0.094569564, 'n03769881 minibus'), (0.077825971, 'n02701002 ambulance')]
3
predicted class is: 555
output label: n03345487 fire engine, fir